In [1]:
%matplotlib nbagg

In [2]:
import os
import contextlib
import glob

import matplotlib.pyplot as plt
import numpy as np
import hyperspy.api as hs
import pyxem
import mrcfile
import stemtool
import pims

import libertem
import libertem.api as lt
from libertem.executor.inline import InlineJobExecutor
from libertem.udf.base import UDF
from libertem.common.buffers import reshaped_view

/Users/weber/.conda/envs/libertem37/lib/python3.7/site-packages/pyUSID/viz/__init__.py:18: FutureWarning: Please use sidpy.viz.plot_utils instead of pyUSID.viz.plot_utils. pyUSID.plot_utils will be removed in a future release of pyUSID
  FutureWarning)


In [3]:
libertem.__version__

'0.6.0.dev0'

In [4]:
sourcefolder = '/cachedata/users/clausen/libertem-test-data/'
targetfolder = '/cachedata/users/weber/libertem-test-data-raw/'
assert os.path.exists(sourcefolder)
assert os.path.exists(targetfolder)

In [5]:
@contextlib.contextmanager
def targetmap(targetfolder, relpath, shape, dtype):
    targetname = os.path.join(targetfolder, f'{relpath}_{shape}_{dtype}.raw')
    os.makedirs(os.path.dirname(targetname), exist_ok=True)
    target = np.memmap(targetname, mode='w+', shape=shape, dtype=dtype)
    yield target
    del target

In [6]:
for path in ['default.blo', 'default.ser']:
    print(path)
    data = hs.load(os.path.join(sourcefolder, path))
    with targetmap(targetfolder, path, data.data.shape, data.data.dtype) as target:
        target[:] = data.data

default.blo
default.ser


In [7]:
relpath = 'default.mib'
assert os.path.exists(os.path.join(sourcefolder, relpath))
data = pyxem.utils.io_utils.load_mib(os.path.join(sourcefolder, relpath))
shape = (32, 32, 256, 256)
with targetmap(targetfolder, relpath, shape, data.data.dtype) as target:
    target[:] = data.data.reshape(shape).compute()

This mib file appears to be TEM data. The stack is returned with no reshaping.


In [8]:
shape = (10, 3838, 3710)
with targetmap(targetfolder, 'dm/2018-7-17 15_29.dm4', shape, 'float32') as target:
    for i, f in enumerate(sorted(glob.glob(os.path.join(sourcefolder, 'dm/*.dm4')))):
        data = hs.load(f)
        print(i, f, data.data.shape)
        target[i] = data.data

0 /cachedata/users/clausen/libertem-test-data/dm/2018-7-17 15_29_0000.dm4 (3838, 3710)
1 /cachedata/users/clausen/libertem-test-data/dm/2018-7-17 15_29_0001.dm4 (3838, 3710)
2 /cachedata/users/clausen/libertem-test-data/dm/2018-7-17 15_29_0002.dm4 (3838, 3710)
3 /cachedata/users/clausen/libertem-test-data/dm/2018-7-17 15_29_0003.dm4 (3838, 3710)
4 /cachedata/users/clausen/libertem-test-data/dm/2018-7-17 15_29_0004.dm4 (3838, 3710)
5 /cachedata/users/clausen/libertem-test-data/dm/2018-7-17 15_29_0005.dm4 (3838, 3710)
6 /cachedata/users/clausen/libertem-test-data/dm/2018-7-17 15_29_0006.dm4 (3838, 3710)
7 /cachedata/users/clausen/libertem-test-data/dm/2018-7-17 15_29_0007.dm4 (3838, 3710)
8 /cachedata/users/clausen/libertem-test-data/dm/2018-7-17 15_29_0008.dm4 (3838, 3710)
9 /cachedata/users/clausen/libertem-test-data/dm/2018-7-17 15_29_0009.dm4 (3838, 3710)


In [9]:
shape = (40, 3838, 3710)
with targetmap(targetfolder, 'dm/3D/alpha-50.dm3', shape, 'float32') as target:
    for i, f in enumerate(sorted(glob.glob(os.path.join(sourcefolder, 'dm/3D/*.dm3')))):
        data = hs.load(f)
        print(i, f, data.data.shape)
        target[i*20:(i+1)*20] = data.data

0 /cachedata/users/clausen/libertem-test-data/dm/3D/alpha-50_obj.dm3 (20, 3838, 3710)
1 /cachedata/users/clausen/libertem-test-data/dm/3D/alpha-50_ref.dm3 (20, 3838, 3710)


In [10]:
class ExportUDF(UDF):
    def __init__(self, targetmap):
        super().__init__(targetmap=targetmap)
        
    def get_preferred_input_dtype(self):
        return UDF.USE_NATIVE_DTYPE
    
    def get_result_buffers(self):
        return {}        
    
    def process_tile(self, tile):
        p = self.params
        shape = self.meta.dataset_shape
        v = reshaped_view(p.targetmap, (np.prod(shape.nav), ) + tuple(shape.sig))
        v[self.meta.slice.get()] = tile

In [11]:
# K2IS uses LiberTEM to generate the reference for now since there are only two alternative readers available: Written
# in Matlab by Colin Ophus, and integrated in GMS.

relpath = 'Capture52/Capture52_.gtg'
ctx = lt.Context(executor=InlineJobExecutor())
ds = ctx.load('auto', path=os.path.join(sourcefolder, relpath))
with targetmap(
        targetfolder=targetfolder,
        relpath=relpath,
        shape=tuple(ds.shape),
        dtype=ds.dtype) as target:
    ctx.run_udf(udf=ExportUDF(targetmap=target), dataset=ds, progress=True)

/cachedata/users/weber/LiberTEM-uellue/src/libertem/utils/devices.py:19: RuntimeWarning: ImportError('CuPy is not correctly installed.\n\nIf you are using wheel distribution (cupy-cudaXX), make sure that the version of CuPy you installed matches with the version of CUDA on your host.\nAlso, confirm that only one CuPy package is installed:\n  $ pip freeze\n\nIf you are building CuPy from source, please check your environment, uninstall CuPy and reinstall it with:\n  $ pip install cupy --no-cache-dir -vvvv\n\nCheck the Installation Guide for details:\n  https://docs.cupy.dev/en/latest/install.html\n\noriginal error: libcudart.so.10.1: cannot open shared object file: No such file or directory')
  warnings.warn(repr(e), RuntimeWarning)
100%|██████████| 17/17 [00:26<00:00,  1.54s/it]


In [12]:
relpath = 'frms6/C16_15_24_151203_019.hdr'
ctx = lt.Context(executor=InlineJobExecutor())
ds = ctx.load('frms6', path=os.path.join(sourcefolder, relpath), enable_offset_correction=False)
ds_corr = ctx.load('frms6', path=os.path.join(sourcefolder, relpath), enable_offset_correction=True)
assert tuple(ds.shape) == (256, 256, 264, 264)

def unfold_stemtool_frms6(data):
    return np.moveaxis(  # undo the transpose that pycroscopy does
        np.repeat(  # unbinning 4x in x direction
            # invert lower half and attach right of upper half
            # The detector consists of two chips that are arranged head-to-head
            # The outputs of the two chips are just concatenated in the file, while LiberTEM
            # re-assembles the data taking the spatial relation into account
            np.concatenate((data[:264], np.flip(data[264:], axis=(0, 1,))), axis=1),
            4, axis=1  # repeat options unbinning x
        ),
        (0, 1, 2), (2, 1, 0)  # moveaxis options undo transpose
    )


def read_contents(fname):
    raw_shape = stemtool.util.pnccd.Frms6Reader.getDataShape(fname)
    frame_count = raw_shape[-1]
    return stemtool.util.pnccd.Frms6Reader.readData(
        fname,
        image_range=(0,frame_count),
        pixels_x = raw_shape[0],
        pixels_y = raw_shape[1]
    )

with targetmap(
        targetfolder=targetfolder,
        relpath=relpath,
        shape=tuple(ds.shape),
        dtype='float32') as target:
    # stemtool opens data as 3D stack, therefore flatten nav
    v = reshaped_view(target, (np.prod(ds.shape.nav), ) + tuple(ds.shape.sig))
    
    fname = os.path.join(sourcefolder, f'frms6/C16_15_24_151203_019_000.frms6')
    darkref_raw = read_contents(fname)
    darkref = np.mean(unfold_stemtool_frms6(darkref_raw), axis=0).astype(np.float32)
    
    # fig, axes = plt.subplots(1, 3)
    # axes[0].imshow(ds_corr._get_dark_frame())
    # axes[1].imshow(darkref)
    # axes[2].imshow(ds_corr._get_dark_frame() - darkref)
    
    offset = 0
    # The first file contains a zero reference    
    for i in range(1, 4):
        fname = os.path.join(sourcefolder, f'frms6/C16_15_24_151203_019_00{i}.frms6')
        data = read_contents(fname)        
        frame_count = data.shape[-1]
        v[offset:offset+frame_count] = unfold_stemtool_frms6(data)
        offset += frame_count
    

    # Consistency check since the reader above is not trivial
    for count in range(10):
        # fig, axes = plt.subplots(2, 3)
        i = np.random.randint(0, np.prod(ds.shape.nav))
        y = i // 256
        x = i % 256
        a = ctx.create_pick_analysis(dataset=ds, y=y, x=x)
        res = ctx.run(a)
        a_corr = ctx.create_pick_analysis(dataset=ds_corr, y=y, x=x)
        res_corr = ctx.run(a_corr)
        # axes[0, 0].imshow(res['intensity'].raw_data)
        # axes[0, 1].imshow(target[y, x])
        # axes[0, 2].imshow(res['intensity'].raw_data - target[y, x])
        
        # axes[1, 0].imshow(res_corr['intensity'].raw_data)
        # axes[1, 1].imshow(target[y, x] - darkref)
        # axes[1, 2].imshow(res_corr['intensity'].raw_data - target[y, x] + darkref)
        
        assert np.allclose(res['intensity'].raw_data, target[y, x])
        assert np.allclose(res_corr['intensity'].raw_data, target[y, x] - darkref)

In [13]:
for size in (4, 256):
    relpath = f'EMPAD/scan_11_x{size}_y{size}.raw'
    shape = (size, size, 128, 128)
    dtype = 'float32'
    with targetmap(
            targetfolder=targetfolder,
            relpath=relpath,
            shape=shape,
            dtype=dtype) as target:
        source = np.memmap(os.path.join(sourcefolder, relpath), shape=(size, size, 130, 128), dtype=dtype, mode='r')
        target[:] = source[:, :, :128, :]

In [14]:
relpath = 'mrc/20200821_92978_movie.mrc'
mrc = mrcfile.open(os.path.join(sourcefolder, relpath))
with targetmap(
        targetfolder=targetfolder,
        relpath=relpath,
        shape=mrc.data.shape,
        dtype=mrc.data.dtype) as target:
    target[:] = mrc.data

In [15]:
relpath = 'default.seq'
images = pims.open(os.path.join(sourcefolder, relpath))

In [16]:
np.array(images)

array([[[ 393,  419,  393, ...,  413,  346,  397],
        [ 342,  396,  396, ...,  403,  400,  415],
        [ 326,  375,  343, ...,  343,  330,  394],
        ...,
        [ 990, 1000,  846, ...,  840,  770, 1002],
        [ 789, 1142,  664, ...,  671,  684,  836],
        [ 759,  958, 1156, ...,  691,  936,  847]],

       [[ 391,  420,  391, ...,  414,  347,  399],
        [ 345,  396,  396, ...,  402,  400,  418],
        [ 327,  373,  344, ...,  341,  333,  394],
        ...,
        [ 704,  862,  741, ...,  895,  645,  850],
        [ 566,  792,  891, ...,  709,  709,  826],
        [ 721,  639,  963, ...,  671,  899,  853]],

       [[ 392,  420,  394, ...,  415,  347,  394],
        [ 342,  396,  396, ...,  404,  402,  417],
        [ 331,  376,  342, ...,  340,  334,  395],
        ...,
        [ 618,  670,  882, ...,  727,  866,  704],
        [ 639,  929,  654, ..., 1124,  908,  833],
        [ 725,  815,  647, ...,  997,  986,  848]],

       ...,

       [[ 392,  421,  39